## Exercise 1

My own function "interp" is shown below. With a test on a selected array, my function takes over three times as long to run as the NumPy interpolation function.

In [10]:
from numba import jit
import numpy as np

def interp(x, xp, fp):
    """xp and yp must be given in ascending order of xp
    with the respective yp in the same position
    yp and xp must be entered as lists
    x must be entered as a list of locations to interpolate
    output will be a list of interpolated values of x
    x must also be between values in xp"""
    
    output = np.zeros(len(x))
    vals = len(fp)
    i = 0
    while i < len(x):
        if x[i] in xp:
            pos = xp.tolist().index(x[i])
            output[i] = fp[i]
            i += 1
        elif x[i] < min(xp):
            output[i] = fp[0]
            i += 1
        elif x[i] > max(xp):
            output[i] = fp[vals]
            i += 1
        else:
            closest = xp[0]
            dist_close = abs(xp[0]-x[i])
            for xp_val in xp:
                dist = abs(xp_val - x[i])
                if dist < dist_close:
                    dist_close = dist
                    closest = xp_val
            pos = xp.tolist().index(closest)
            if x[i] > closest:
                x_lower = closest
                y_lower = fp[pos]
                x_upper = xp[pos+1]
                y_upper = fp[pos+1]   
            else:
                x_lower = xp[pos-1]
                y_lower = fp[pos-1]
                x_upper = closest
                y_upper = fp[pos]
            slope = (y_upper-y_lower)/(x_upper-x_lower)
            x_interp = y_lower + (x[i]-x_lower)*slope
            output[i] = x_interp
            i += 1
    return output
            

In [11]:
a = np.array([2,3,5])
b = np.array([1,4,5,6])
c = np.array([0,1,3,4])

print(interp(a,b,c))
%timeit interp(a,b,c)

[ 0.33333333  0.66666667  3.        ]
The slowest run took 5.87 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 35.2 µs per loop


In [12]:
interp_numba = jit(interp)
%timeit interp_numba(a,b,c)

The slowest run took 7650.04 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 127 µs per loop


## Exercise 2

I produce a modified version of the coleman_egm function that uses my own interpolation function and Numba. This new function actually takes much longer to run than the original function, which may be due to the loops in my program or due to the time that it takes to use Numba.

In [14]:
import numpy as np

def coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    """
    The approximate Coleman operator, updated using the endogenous grid
    method.  
    
    Parameters
    ----------
    g : function
        The current guess of the policy function
    k_grid : array_like(float, ndim=1)
        The set of *exogenous* grid points, for capital k = y - c
    beta : scalar
        The discount factor
    u_prime : function
        The derivative u'(c) of the utility function
    u_prime_inv : function
        The inverse of u' (which exists by assumption)
    f : function
        The production function f(k)
    f_prime : function
        The derivative f'(k)
    shocks : numpy array
        An array of draws from the shock, for Monte Carlo integration (to
        compute expectations).

    """

    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: np.interp(x, y, c)
    return Kg

def coleman_egm_numba(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    """
    The approximate Coleman operator, updated using the endogenous grid
    method.  
    
    Parameters
    ----------
    g : function
        The current guess of the policy function
    k_grid : array_like(float, ndim=1)
        The set of *exogenous* grid points, for capital k = y - c
    beta : scalar
        The discount factor
    u_prime : function
        The derivative u'(c) of the utility function
    u_prime_inv : function
        The inverse of u' (which exists by assumption)
    f : function
        The production function f(k)
    f_prime : function
        The derivative f'(k)
    shocks : numpy array
        An array of draws from the shock, for Monte Carlo integration (to
        compute expectations).

    """

    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: interp(x, y, c)
    return Kg


import matplotlib.pyplot as plt
import quantecon as qe

class LogLinearOG:
    """
    Log linear optimal growth model, with log utility, CD production and
    multiplicative lognormal shock, so that

        y = f(k, z) = z k^alpha

    with z ~ LN(mu, s).

    The class holds parameters and true value and policy functions.
    """

    def __init__(self, alpha=0.4, beta=0.96, mu=0, s=0.1):

        self.alpha, self.beta, self.mu, self.s = alpha, beta, mu, s 

        # == Some useful constants == #
        self.ab = alpha * beta
        self.c1 = np.log(1 - self.ab) / (1 - beta)
        self.c2 = (mu + alpha * np.log(self.ab)) / (1 - alpha)
        self.c3 = 1 / (1 - beta)
        self.c4 = 1 / (1 - self.ab)

    def u(self, c):
        " Utility "
        return np.log(c)

    def u_prime(self, c):
        return 1 / c

    def f(self, k):
        " Deterministic part of production function.  "
        return k**self.alpha

    def f_prime(self, k):
        return self.alpha * k**(self.alpha - 1)

    def c_star(self, y):
        " True optimal policy.  "
        return (1 - self.alpha * self.beta) * y

    def v_star(self, y):
        " True value function. "
        return self.c1 + self.c2 * (self.c3 - self.c4) + self.c4 * np.log(y)

lg = LogLinearOG()

# == Unpack parameters / functions for convenience == #
alpha, beta, mu, s = lg.alpha, lg.beta, lg.mu, lg.s
v_star, c_star = lg.v_star, lg.c_star
u, u_prime, f, f_prime = lg.u, lg.u_prime, lg.f, lg.f_prime

grid_max = 4         # Largest grid point, exogenous grid
grid_size = 200      # Number of grid points
shock_size = 250     # Number of shock draws in Monte Carlo integral

k_grid = np.linspace(1e-5, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))

c_star_new = coleman_egm(c_star,
            k_grid, beta, u_prime, u_prime, f, f_prime, shocks)

alpha = 0.65
beta = 0.95
mu = 0
s = 0.1
grid_min = 1e-6
grid_max = 4
grid_size = 200
shock_size = 250

gamma = 1.5   # Preference parameter
gamma_inv = 1 / gamma

def f(k):
    return k**alpha

def f_prime(k):
    return alpha * k**(alpha - 1)

def u(c):
    return (c**(1 - gamma) - 1) / (1 - gamma)

def u_prime(c):
    return c**(-gamma)

def u_prime_inv(c):
    return c**(-gamma_inv)

k_grid = np.linspace(grid_min, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))

## Let's make convenience functions based around these primitives

def crra_coleman(g):
    return coleman_operator(g, k_grid, beta, u_prime, f, f_prime, shocks)

def crra_coleman_egm(g):
    return coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

def crra_coleman_egm_numba(g):
    return coleman_egm_numba(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

sim_length = 20

print("Timing policy function iteration with endogenous grid")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm(g)
    g = new_g
qe.util.toc()

print("Timing policy function iteration with endogenous grid and Numba")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm_numba(g)
    g = new_g
qe.util.toc()

Timing policy function iteration with endogenous grid
TOC: Elapsed: 0.16455602645874023 seconds.
Timing policy function iteration with endogenous grid and Numba
TOC: Elapsed: 147.87833404541016 seconds.


147.87833404541016